# 🚀 Pipeline - Modelo LSTM - Projeto USP-IC-SIN5006
Este notebook executa o modelo LSTM sobre todas as bases localizadas na pasta `datasets` do Google Drive.
Os resultados (previsões, gráficos e métricas) serão salvos automaticamente na pasta `outputs`.
---


In [3]:

# 🔧 Instalação de pacotes necessários
!pip install tensorflow scikit-learn pandas matplotlib seaborn plotly --quiet

# 🔗 Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 🗂️ Definir os caminhos das pastas
base_path = '/content/drive/MyDrive/USP-IC-SIN5006/'
datasets_path = base_path + 'datasets/'
outputs_path = base_path + 'outputs/'


Mounted at /content/drive


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, GRU
import os
import glob

datasets = glob.glob(datasets_path + '*.csv')
print(f'🗂️ Bases encontradas: {datasets}')

def create_sequences(data, look_back=30):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:(i + look_back)])
        y.append(data[i + look_back])
    return np.array(X), np.array(y)

def build_model(input_shape, model_type, dropout_rate=0.0):
    model = Sequential()
    if model_type == 'LSTM':
        model.add(LSTM(50, activation='tanh', input_shape=input_shape))
        if dropout_rate > 0:
            model.add(Dropout(dropout_rate))
    elif model_type == 'GRU':
        model.add(GRU(50, activation='tanh', input_shape=input_shape))
        if dropout_rate > 0:
            model.add(Dropout(dropout_rate))
    elif model_type == 'MLP':
        model.add(Dense(64, activation='relu', input_shape=(input_shape[0],)))
        model.add(Dense(32, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

def evaluate_model(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred) ** 0.5
    r2 = r2_score(y_true, y_pred)
    return {'MAE': mae, 'RMSE': rmse, 'R2': r2}

results = []
look_back = 30
model_type = 'LSTM' # Define o tipo de modelo a ser usado

for dataset_path in datasets:
    dataset_name = os.path.basename(dataset_path).replace('.csv','')
    print(f"📈 Processando dataset: {dataset_name}")

    df = pd.read_csv(dataset_path)
    # Select only numerical columns for scaling
    values = df.select_dtypes(include=np.number).values

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled = scaler.fit_transform(values)

    X, y = create_sequences(scaled, look_back)

    # --- Divisão em treino e teste para séries temporais ---
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[0:train_size], X[train_size:len(X)]
    y_train, y_test = y[0:train_size], y[train_size:len(y)]
    # --- Fim da divisão ---

    X_train_model = X_train.reshape((X_train.shape[0], X_train.shape[1], X_train.shape[2])) if model_type in ['LSTM', 'GRU'] else X_train.reshape((X_train.shape[0], X_train.shape[1]))
    X_test_model = X_test.reshape((X_test.shape[0], X_test.shape[1], X_test.shape[2])) if model_type in ['LSTM', 'GRU'] else X_test.reshape((X_test.shape[0], X_test.shape[1]))


    input_shape = (X.shape[1], X.shape[2]) if model_type in ['LSTM', 'GRU'] else (X.shape[1],)

    # --- Ajustes específicos para o dataset 'bar' ---
    epochs = 50
    dropout_rate = 0.0
    if dataset_name == 'bar':
        epochs = 100 # Aumenta as épocas
        dropout_rate = 0.2 # Adiciona Dropout
        print(f"✨ Aplicando ajustes para o dataset '{dataset_name}': Épocas={epochs}, Dropout={dropout_rate}")
    # --- Fim dos ajustes específicos ---

    model = build_model(input_shape, model_type, dropout_rate=dropout_rate)

    history = model.fit(
        X_train_model, # Treina no conjunto de treino
        y_train,
        epochs=epochs, # Usa o número de épocas ajustado
        batch_size=16,
        verbose=0
    )

    y_pred = model.predict(X_test_model) # Faz a previsão no conjunto de teste

    # --- Correção da inversão de escala ---
    # Ensure y_test and y_pred have shape (n_samples, 1) for consistency before inverse transform
    y_test_reshaped = y_test.reshape(-1, 1)
    y_pred_reshaped = y_pred.reshape(-1, 1)


    # Crie arrays temporárias com o número total de features (scaled.shape[1]) para a inversão
    # e coloque os valores preditos/reais na posição da coluna alvo (assumindo a primeira)
    y_test_temp = np.zeros((len(y_test_reshaped), scaled.shape[1]))
    y_test_temp[:, 0] = y_test_reshaped[:, 0] # Place y_test values in the first column

    y_pred_temp = np.zeros((len(y_pred_reshaped), scaled.shape[1]))
    y_pred_temp[:, 0] = y_pred_reshaped[:, 0] # Place y_pred values in the first column


    # Inverta a escala e selecione apenas a coluna alvo (a primeira)
    y_true_inv = scaler.inverse_transform(y_test_temp)[:, 0]
    y_pred_inv = scaler.inverse_transform(y_pred_temp)[:, 0]
    # --- Fim da correção ---


    metrics = evaluate_model(y_true_inv, y_pred_inv) # Avalia usando os arrays 1D inversos
    metrics['Dataset'] = dataset_name
    metrics['Model'] = model_type
    results.append(metrics)

    plt.figure(figsize=(10,5))
    plt.plot(y_true_inv, label='Real') # Plota o conjunto de teste real (array 1D)
    plt.plot(y_pred_inv, label='Previsto') # Plota a previsão no conjunto de teste (array 1D)
    plt.title(f'Previsão - {dataset_name} - {model_type} (Teste)')
    plt.legend()
    plot_file = outputs_path + f'plot_{dataset_name}_{model_type}_test.png' # Salva com nome indicando teste
    plt.savefig(plot_file)
    plt.show()

    df_out = pd.DataFrame({
        'Real': y_true_inv, # Usa o array 1D diretamente
        'Previsto': y_pred_inv # Usa o array 1D diretamente
    })
    output_name = outputs_path + f'resultado_{dataset_name}_{model_type}_test.csv' # Salva com nome indicando teste
    df_out.to_csv(output_name, index=False)
    print(f"✅ Resultado salvo: {output_name}")
    print(f"🖼️ Gráfico salvo: {plot_file}")

df_results = pd.DataFrame(results)
print(df_results)
df_results.to_csv(outputs_path + 'Resultados_Consolidados_LSTM_test_evaluation.csv', index=False) # Atualiza o nome do arquivo consolidado
print("📄 Resultados consolidados salvos em 'Resultados_Consolidados_LSTM_test_evaluation.csv'")

🗂️ Bases encontradas: ['/content/drive/MyDrive/USP-IC-SIN5006/datasets/cod.csv', '/content/drive/MyDrive/USP-IC-SIN5006/datasets/coa.csv', '/content/drive/MyDrive/USP-IC-SIN5006/datasets/par.csv', '/content/drive/MyDrive/USP-IC-SIN5006/datasets/bar.csv', '/content/drive/MyDrive/USP-IC-SIN5006/datasets/bjc.csv', '/content/drive/MyDrive/USP-IC-SIN5006/datasets/mna.csv', '/content/drive/MyDrive/USP-IC-SIN5006/datasets/eir.csv', '/content/drive/MyDrive/USP-IC-SIN5006/datasets/mnc.csv', '/content/drive/MyDrive/USP-IC-SIN5006/datasets/sgc.csv', '/content/drive/MyDrive/USP-IC-SIN5006/datasets/lab.csv', '/content/drive/MyDrive/USP-IC-SIN5006/datasets/iau.csv', '/content/drive/MyDrive/USP-IC-SIN5006/datasets/tef.csv']
📈 Processando dataset: cod


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step


ValueError: Found input variables with inconsistent numbers of samples: [5944, 743]